<a href="https://colab.research.google.com/github/thiago-lpereira/2H_ExGIThiago/blob/main/VisualizaData_DataViews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import joblib
import warnings

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
warnings.simplefilter(action='ignore', category=FutureWarning)

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.initializers import GlorotUniform

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_breast_cancer

def plot_training_history(history, dataset_name):

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

    ax1.plot(history.history['accuracy'])
    ax1.plot(history.history['val_accuracy'])
    ax1.set_title(f'Model Accuracy ({dataset_name})')
    ax1.set_ylabel('Accuracy')
    ax1.set_xlabel('Epoch')
    ax1.legend(['Train', 'Test'], loc='upper left')
    ax1.grid(True, linestyle='--', alpha=0.6)

    ax2.plot(history.history['loss'])
    ax2.plot(history.history['val_loss'])
    ax2.set_title(f'Model Loss ({dataset_name})')
    ax2.set_ylabel('Loss')
    ax2.set_xlabel('Epoch')
    ax2.legend(['Train', 'Test'], loc='upper left')
    ax2.grid(True, linestyle='--', alpha=0.6)

    plt.tight_layout()
    plt.show()

print("\n--- 1. Starting Breast Cancer Dataset Analysis ---")

cancer_data = load_breast_cancer()
X_cancer_df = pd.DataFrame(cancer_data.data, columns=cancer_data.feature_names)
y_cancer = cancer_data.target

X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(
    X_cancer_df, y_cancer, test_size=0.2, random_state=42
)

X_train_c_nonscaled = X_train_c.copy()
X_test_c_nonscaled = X_test_c.copy()

scaler_c = StandardScaler()
X_train_c = scaler_c.fit_transform(X_train_c)
X_test_c = scaler_c.transform(X_test_c)

model_cancer = Sequential([
    Dense(16, input_dim=X_train_c.shape[1], activation='relu', kernel_initializer=GlorotUniform(seed=69)),
    Dense(8, activation='relu', kernel_initializer=GlorotUniform(seed=69)),
    Dense(1, activation='sigmoid', kernel_initializer=GlorotUniform(seed=69))
])

model_cancer.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history_cancer = model_cancer.fit(
    X_train_c, y_train_c,
    validation_data=(X_test_c, y_test_c),
    epochs=100,
    verbose=0
)

loss_c, accuracy_c = model_cancer.evaluate(X_test_c, y_test_c, verbose=0)
print(f"Final Test Accuracy (Breast Cancer): {accuracy_c * 100:.2f}%")

plot_training_history(history_cancer, "Breast Cancer Dataset")

print("Serializing Breast Cancer components...")
model_cancer.save('breast_cancer_model.h5')
joblib.dump(scaler_c, 'breast_cancer_scaler.pkl')
np.savez_compressed(
    'breast_cancer_datasets.npz',
    X_train=X_train_c_nonscaled.to_numpy(),
    X_test=X_test_c_nonscaled.to_numpy(),
    y_train=y_train_c,
    y_test=y_test_c,
    columns=X_train_c_nonscaled.columns.tolist()
)
print("Breast Cancer components serialized.")


print("\n--- 2. Starting Titanic Dataset Analysis ---")

df_titanic = sns.load_dataset('titanic')

df_titanic['age'].fillna(df_titanic['age'].median(), inplace=True)
df_titanic['embarked'].fillna(df_titanic['embarked'].mode()[0], inplace=True)
df_titanic.drop(columns=['deck', 'embark_town', 'alive'], inplace=True)
df_titanic['sex'] = df_titanic['sex'].map({'male': 0, 'female': 1})
df_titanic = pd.get_dummies(df_titanic, columns=['embarked', 'class', 'who', 'adult_male', 'alone'], drop_first=True, dtype=int)

X_titanic = df_titanic.drop('survived', axis=1)
y_titanic = df_titanic['survived']

X_train_t, X_test_t, y_train_t, y_test_t = train_test_split(
    X_titanic, y_titanic, test_size=0.2, random_state=42
)

X_train_t_nonscaled = X_train_t.copy()
X_test_t_nonscaled = X_test_t.copy()

scaler_t = StandardScaler()
X_train_t = scaler_t.fit_transform(X_train_t)
X_test_t = scaler_t.transform(X_test_t)

model_titanic = Sequential([
    Dense(16, input_dim=X_train_t.shape[1], activation='relu', kernel_initializer=GlorotUniform(seed=469)),
    Dense(8, activation='relu', kernel_initializer=GlorotUniform(seed=469)),
    Dense(1, activation='sigmoid', kernel_initializer=GlorotUniform(seed=469))
])
model_titanic.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

history_titanic = model_titanic.fit(
    X_train_t, y_train_t,
    validation_data=(X_test_t, y_test_t),
    epochs=100,
    verbose=0
)

loss_t, accuracy_t = model_titanic.evaluate(X_test_t, y_test_t, verbose=0)
print(f"Final Test Accuracy (Titanic): {accuracy_t * 100:.2f}%")

plot_training_history(history_titanic, "Titanic Dataset")

print("Serializing Titanic components...")
model_titanic.save('titanic_model.h5')
joblib.dump(scaler_t, 'titanic_scaler.pkl')
np.savez_compressed(
    'titanic_datasets.npz',
    X_train=X_train_t_nonscaled.to_numpy(),
    X_test=X_test_t_nonscaled.to_numpy(),
    y_train=y_train_t.to_numpy(),
    y_test=y_test_t.to_numpy(),
    columns=X_train_t_nonscaled.columns.tolist()
)
print("Titanic components serialized.")


print("\n--- 3. Starting Adult (Income) Dataset Analysis ---")

url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss',
    'hours-per-week', 'native-country', 'income'
]
df_adult = pd.read_csv(url, header=None, names=columns, na_values=' ?', skipinitialspace=True)

df_adult.dropna(inplace=True)

df_adult['income'] = df_adult['income'].map({'<=50K': 0, '>50K': 1})

df_adult = pd.get_dummies(df_adult, drop_first=True, dtype=int)

X_adult = df_adult.drop('income', axis=1)
y_adult = df_adult['income']

X_train_a, X_test_a, y_train_a, y_test_a = train_test_split(
    X_adult, y_adult, test_size=0.2, random_state=42
)

X_train_a_nonscaled = X_train_a.copy()
X_test_a_nonscaled = X_test_a.copy()

scaler_a = StandardScaler()
X_train_a = scaler_a.fit_transform(X_train_a)
X_test_a = scaler_a.transform(X_test_a)

model_adult = Sequential([
    Dense(32, input_dim=X_train_a.shape[1], activation='relu', kernel_initializer=GlorotUniform(seed=777)),
    Dense(16, activation='relu', kernel_initializer=GlorotUniform(seed=777)),
    Dense(1, activation='sigmoid', kernel_initializer=GlorotUniform(seed=777))
])
model_adult.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history_adult = model_adult.fit(
    X_train_a, y_train_a,
    validation_data=(X_test_a, y_test_a),
    epochs=50,
    verbose=0
)

loss_a, accuracy_a = model_adult.evaluate(X_test_a, y_test_a, verbose=0)
print(f"Final Test Accuracy (Adult Income): {accuracy_a * 100:.2f}%")

plot_training_history(history_adult, "Adult Income Dataset")

print("Serializing Adult Income components...")
model_adult.save('adult_model.h5')
joblib.dump(scaler_a, 'adult_scaler.pkl')
np.savez_compressed(
    'adult_datasets.npz',
    X_train=X_train_a_nonscaled.to_numpy(),
    X_test=X_test_a_nonscaled.to_numpy(),
    y_train=y_train_a.to_numpy(),
    y_test=y_test_a.to_numpy(),
    columns=X_train_a_nonscaled.columns.tolist()
)
print("Adult Income components serialized.")


# --- 4. Deserialization and Verification ---

print("\n\n--- 4. Starting Deserialization and Verification ---")

print("\n--- Verifying Breast Cancer ---")
loaded_model_c = load_model('breast_cancer_model.h5')
loaded_model_c.summary()

loaded_scaler_c = joblib.load('breast_cancer_scaler.pkl')
data_c = np.load('breast_cancer_datasets.npz', allow_pickle=True)

loaded_columns_c = data_c['columns']
X_train_loaded_c = pd.DataFrame(data_c['X_train'], columns=loaded_columns_c)
X_test_loaded_c = pd.DataFrame(data_c['X_test'], columns=loaded_columns_c)
y_train_loaded_c = data_c['y_train']
y_test_loaded_c = data_c['y_test']
print(f"DataFrame shape: {X_test_loaded_c.shape}") # Isso agora deve mostrar (num_rows, 14)


X_test_scaled_loaded_c = loaded_scaler_c.transform(X_test_loaded_c)
loss_loaded_c, accuracy_loaded_c = loaded_model_c.evaluate(X_test_scaled_loaded_c, y_test_loaded_c, verbose=0)
print(f"Accuracy of LOADED model (Breast Cancer): {accuracy_loaded_c * 100:.2f}%")




loaded_model = load_model('titanic_model.h5')
print("Model loaded successfully.")
loaded_model.summary()

loaded_scaler = joblib.load('titanic_scaler.pkl')
print("Scaler loaded successfully.")





data = np.load('titanic_datasets.npz', allow_pickle=True)

loaded_columns = data['columns']

X_train_loaded = pd.DataFrame(data['X_train'], columns=loaded_columns)
X_test_loaded = pd.DataFrame(data['X_test'], columns=loaded_columns)
y_train_loaded = data['y_train']
y_test_loaded = data['y_test']
print("Datasets loaded successfully.")
print(f"DataFrame shape: {X_test_loaded.shape}") # Isso agora deve mostrar (num_rows, 14)

X_test_scaled_loaded = loaded_scaler.transform(X_test_loaded)
loss_loaded, accuracy_loaded = loaded_model.evaluate(X_test_scaled_loaded, y_test_loaded, verbose=0)
print(f"Accuracy of the loaded model on the test data: {accuracy_loaded * 100:.2f}%")


print("Deserialization and verification complete.")



print("\n--- Verifying Adult Income ---")
loaded_model_a = load_model('adult_model.h5')
loaded_model_a.summary()
loaded_scaler_a = joblib.load('adult_scaler.pkl')
data_a = np.load('adult_datasets.npz', allow_pickle=True)

loaded_columns_a = data_a['columns']
X_train_loaded_a = pd.DataFrame(data_a['X_train'], columns=loaded_columns_a)
X_test_loaded_a = pd.DataFrame(data_a['X_test'], columns=loaded_columns_a)
y_train_loaded_a = data_a['y_train']
y_test_loaded_a = data_a['y_test']
print(f"DataFrame shape: {X_test_loaded_a.shape}") # Isso agora deve mostrar (num_rows, 14)

X_test_scaled_loaded_a = loaded_scaler_a.transform(X_test_loaded_a)
loss_loaded_a, accuracy_loaded_a = loaded_model_a.evaluate(X_test_scaled_loaded_a, y_test_loaded_a, verbose=0)
print(f"Accuracy of LOADED model (Adult Income): {accuracy_loaded_a * 100:.2f}%")


print("\n--- Deserialization and all verifications complete. ---")

In [25]:
# %%
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score
from numpy import mean
import joblib
from sklearn.preprocessing import StandardScaler

# %%
# Load the Breast Cancer dataset
data_cancer = load_breast_cancer()
df_cancer = pd.DataFrame(data_cancer.data, columns=data_cancer.feature_names)
df_cancer['target'] = data_cancer.target

# Split data (though not strictly needed for the sampling loop below, good practice)
X_cancer = df_cancer[data_cancer.feature_names]
y_cancer = df_cancer['target']

X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(
    X_cancer, y_cancer,
    test_size=0.2,
    random_state=42,
)

print("X_train_c:", X_train_c.shape)
print("X_test_c: ", X_test_c.shape)
print("y_train_c:", y_train_c.shape)
print("y_test_c: ", y_test_c.shape)

# %%
# Load the model and scaler for Breast Cancer
modelo_cancer = load_model("breast_cancer_model.h5")
scaler_c = joblib.load('breast_cancer_scaler.pkl')

# Predict and evaluate on the scaled test data
X_test_c_scaled = scaler_c.transform(X_test_c)
y_pred_c = modelo_cancer.predict(X_test_c_scaled)

# Convert predictions to binary classes
y_pred_classes_c = (y_pred_c > 0.5).astype("int32")

display(y_pred_c)

# %%
# Calculate accuracy
acertos_c = accuracy_score(y_test_c, y_pred_classes_c)
print("Acurácia:", acertos_c)

# %%
# The following code block is for repeated evaluation on samples of the Breast Cancer dataset
breast_cancer_accuracies = []

for i in range(0,1000):
    # Sample from the original Breast Cancer data
    df_sample_c = df_cancer.sample(n=100, random_state=i, replace=False)

    # Separate features and target for the sample
    X_sample_c = df_sample_c[data_cancer.feature_names]
    y_sample_c = df_sample_c['target']

    # Scale the sampled data
    X_sample_scaled_c = scaler_c.transform(X_sample_c)

    # Evaluate the loaded model on the sampled and scaled data
    loss_c, accuracy_c = modelo_cancer.evaluate(X_sample_scaled_c, y_sample_c, verbose=0)
    breast_cancer_accuracies.append(accuracy_c)


# %%
print("Mean Accuracy (Breast Cancer Samples):", mean(breast_cancer_accuracies))

X_train_c: (455, 30)
X_test_c:  (114, 30)
y_train_c: (455,)
y_test_c:  (114,)
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


array([[9.82021391e-01],
       [7.74852094e-09],
       [1.80249008e-05],
       [9.99978065e-01],
       [9.99998629e-01],
       [5.79698894e-17],
       [6.79356609e-14],
       [3.02863633e-03],
       [6.95912480e-01],
       [9.99997199e-01],
       [8.73546660e-01],
       [1.25560269e-04],
       [9.99852836e-01],
       [4.65241494e-04],
       [9.99995768e-01],
       [5.00314854e-07],
       [9.99985456e-01],
       [9.99999702e-01],
       [9.99999940e-01],
       [4.60664701e-11],
       [9.96562779e-01],
       [9.99636173e-01],
       [1.06882005e-14],
       [9.99991715e-01],
       [9.99971449e-01],
       [9.99997199e-01],
       [9.99997020e-01],
       [9.99095678e-01],
       [9.99925256e-01],
       [1.27150968e-09],
       [9.99902785e-01],
       [9.99998987e-01],
       [9.97683465e-01],
       [9.99875307e-01],
       [9.99993265e-01],
       [9.99865949e-01],
       [8.51708557e-03],
       [9.98818994e-01],
       [1.63611826e-08],
       [9.93141532e-01],


Acurácia: 0.9824561403508771
Mean Accuracy (Breast Cancer Samples): 0.9965200033187867


In [23]:
# %%
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score
from numpy import mean
import joblib
from sklearn.preprocessing import StandardScaler


print("\n--- 3. Starting Adult (Income) Dataset Analysis ---")

# Load the Adult Income dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss',
    'hours-per-week', 'native-country', 'income'
]
original_data_adult = pd.read_csv(url, header=None, names=columns, na_values=' ?', skipinitialspace=True)

# Define preprocessing function for Adult Income data
def preprocess_adult_data(df):
    df.dropna(inplace=True)
    df['income'] = df['income'].map({'<=50K': 0, '>50K': 1})
    df = pd.get_dummies(df, drop_first=True, dtype=int)
    return df

# Apply preprocessing to the original Adult Income data for train/test split and sampling
df_adult_processed = preprocess_adult_data(original_data_adult.copy())

X_adult = df_adult_processed.drop('income', axis=1)
y_adult = df_adult_processed['income']

X_train_a, X_test_a, y_train_a, y_test_a = train_test_split(
    X_adult, y_adult, test_size=0.2, random_state=42
)

# Fit scaler on the training data
scaler_a = StandardScaler()
X_train_a_scaled = scaler_a.fit_transform(X_train_a)
X_test_a_scaled = scaler_a.transform(X_test_a)


print("X_train:", X_train_a.shape)
print("X_test: ", X_test_a.shape)
print("y_train:", y_train_a.shape)
print("y_test: ", y_test_a.shape)

# %%
# Load the model and scaler
modelo_adult = load_model("adult_model.h5")
scaler_a = joblib.load('adult_scaler.pkl')

# Predict and evaluate on the scaled test data
y_pred_a = modelo_adult.predict(X_test_a_scaled)

# Convert predictions to binary classes
y_pred_classes_a = (y_pred_a > 0.5).astype("int32")

display(y_pred_a)

# %%
# Calculate accuracy
acertos_a = accuracy_score(y_test_a, y_pred_classes_a)
print("Acurácia:", acertos_a)

# %%
# The following code block is for repeated evaluation on samples of the Adult Income dataset
# It's important to use the correctly processed and scaled Adult Income data here as well.

adult_accuracies = []

for i in range(0,1000):
    # Sample from the processed Adult Income data
    df_sample_processed_a = df_adult_processed.sample(n=100, random_state=i, replace=False)

    # Separate features and target for the sample
    X_sample_a = df_sample_processed_a.drop('income', axis=1)
    y_sample_a = df_sample_processed_a['income']

    # Scale the sampled data using the fitted scaler
    X_sample_scaled_a = scaler_a.transform(X_sample_a)

    # Evaluate the loaded model on the sampled and scaled data
    loss_a, accuracy_a = modelo_adult.evaluate(X_sample_scaled_a, y_sample_a, verbose=0)
    adult_accuracies.append(accuracy_a)

# %%
print("Mean Accuracy (Adult Income Samples):", mean(adult_accuracies))


--- 3. Starting Adult (Income) Dataset Analysis ---


X_train: (26048, 100)
X_test:  (6513, 100)
y_train: (26048,)
y_test:  (6513,)
204/204 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


array([[1.7498140e-03],
       [2.8722030e-01],
       [9.2872447e-01],
       ...,
       [1.0000000e+00],
       [1.5830391e-03],
       [2.3615550e-05]], dtype=float32)

Acurácia: 0.8489175495163519
Mean Accuracy (Adult Income Samples): 0.8786799997091294


In [24]:
# %%
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score
import seaborn as sns
from numpy import mean
import joblib
from sklearn.preprocessing import StandardScaler


print("\n--- 2. Starting Titanic Dataset Analysis ---")

# Load the Titanic dataset
original_data_titanic = sns.load_dataset('titanic')

# Define preprocessing function for Titanic data
def preprocess_titanic_data(df):
    df['age'].fillna(df['age'].median(), inplace=True)
    df['embarked'].fillna(df['embarked'].mode()[0], inplace=True)
    df.drop(columns=['deck', 'embark_town', 'alive'], inplace=True)
    df['sex'] = df['sex'].map({'male': 0, 'female': 1})
    df = pd.get_dummies(df, columns=['embarked', 'class', 'who', 'adult_male', 'alone'], drop_first=True, dtype=int)
    return df

# Apply preprocessing to the original Titanic data for train/test split and sampling
df_titanic_processed = preprocess_titanic_data(original_data_titanic.copy())

X_titanic = df_titanic_processed.drop('survived', axis=1)
y_titanic = df_titanic_processed['survived']

X_train_t, X_test_t, y_train_t, y_test_t = train_test_split(
    X_titanic, y_titanic, test_size=0.2, random_state=42
)

# Fit scaler on the training data
scaler_t = StandardScaler()
X_train_t_scaled = scaler_t.fit_transform(X_train_t)
X_test_t_scaled = scaler_t.transform(X_test_t)


print("X_train:", X_train_t.shape)
print("X_test: ", X_test_t.shape)
print("y_train:", y_train_t.shape)
print("y_test: ", y_test_t.shape)

# %%
# Load the model and scaler for Titanic
modelo_titanic = load_model("titanic_model.h5")
scaler_t = joblib.load('titanic_scaler.pkl')

# Predict and evaluate on the scaled test data
y_pred_t = modelo_titanic.predict(X_test_t_scaled)

# Convert predictions to binary classes
y_pred_classes_t = (y_pred_t > 0.5).astype("int32")

display(y_pred_t)

# %%
# Calculate accuracy
acertos_t = accuracy_score(y_test_t, y_pred_classes_t)
print("Acurácia:", acertos_t)

# %%
# The following code block is for repeated evaluation on samples of the Titanic dataset
# It's important to use the correctly processed and scaled Titanic data here as well.
# Also, the variable for storing accuracies should be used in the mean function.

titanic_accuracies = []

for i in range(0,1000):
    # Sample from the processed Titanic data
    df_sample_processed_t = df_titanic_processed.sample(n=100, random_state=i, replace=False)

    # Separate features and target for the sample
    X_sample_t = df_sample_processed_t.drop('survived', axis=1)
    y_sample_t = df_sample_processed_t['survived']

    # Scale the sampled data using the fitted scaler
    X_sample_scaled_t = scaler_t.transform(X_sample_t)

    # Evaluate the loaded model on the sampled and scaled data
    loss_t, accuracy_t = modelo_titanic.evaluate(X_sample_scaled_t, y_sample_t, verbose=0)
    titanic_accuracies.append(accuracy_t)




# %%
print("Mean Accuracy (Titanic Samples):", mean(titanic_accuracies))


--- 2. Starting Titanic Dataset Analysis ---
X_train: (712, 14)
X_test:  (179, 14)
y_train: (712,)
y_test:  (179,)
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step


array([[0.19601594],
       [0.08616011],
       [0.13387816],
       [0.98827624],
       [0.5630278 ],
       [0.97575647],
       [0.7641969 ],
       [0.15352222],
       [0.72053635],
       [0.952948  ],
       [0.3222808 ],
       [0.0924968 ],
       [0.2522168 ],
       [0.20488204],
       [0.09844542],
       [0.9715794 ],
       [0.26018956],
       [0.76432496],
       [0.07669904],
       [0.36686745],
       [0.13912724],
       [0.29651594],
       [0.42756182],
       [0.13695842],
       [0.1577647 ],
       [0.11506502],
       [0.38174084],
       [0.08768396],
       [0.15058072],
       [0.4225246 ],
       [0.13486668],
       [0.43889967],
       [0.3442332 ],
       [0.410728  ],
       [0.13810253],
       [0.19940655],
       [0.2865437 ],
       [0.7641969 ],
       [0.97326845],
       [0.14056674],
       [0.07987513],
       [0.13526693],
       [0.14094977],
       [0.09358966],
       [0.7253473 ],
       [0.09371332],
       [0.13554417],
       [0.141

Acurácia: 0.8212290502793296
Mean Accuracy (Titanic Samples): 0.8396499986052514
